# 目录
## 1. 导入模块
## 2. GPU的配置策略（因为只有一个gpu，所以就虚拟4个gpu测试）
## 3. 测试

## 1. 导入模块

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow as tf

import sys
import os
import time
import datetime

for module in [np, pd, sklearn, mpl, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
matplotlib 3.1.2
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. GPU的配置策略（因为只有一个gpu，所以就虚拟4个gpu测试）

In [2]:
'''
tf.debugging.set_log_device_placement(True)  # 打印设备运行日志
tf.config.set_soft_device_placement(True)  # 把运算放在合适的设备运行
GPUS = tf.config.experimental.list_physical_devices("GPU") # 获取物理GPU
for gpu in GPUS:
    # 设置GPU内存自增长模式，需要多少，就用多少， 这一行代码必须要在程序刚开始运行的时候运行
    tf.config.experimental.set_memory_growth(gpu, True)  
print("Physical GPU: {}".format(len(GPUS)))

logical_gpus = tf.config.experimental.list_logical_devices("GPU") # 获取逻辑GPU，逻辑GPU就像电脑磁盘分区，默认是在第0个逻辑分区训练
print("Logical GPU: {}".format(len(logical_gpus)))

'''
# 虚拟多个gpu，用来验证。。。如果有多个物理gpu，就用上面的代码

tf.debugging.set_log_device_placement(True)  # 打印设备运行日志
tf.config.set_soft_device_placement(True)  # 把运算放在合适的设备运行
GPUS = tf.config.experimental.list_physical_devices("GPU") # 获取物理GPU
tf.config.experimental.set_visible_devices(GPUS[0], "GPU") # 设置该程序可访问的GPU，默认是在第0个gpu上训练
tf.config.experimental.set_virtual_device_configuration(  # 对某一个物理gpu设置逻辑分区， 默认一个物理gpu对应一个逻辑gpu
    GPUS[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2072),
     tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2072),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2072),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2072)]
)

print("Physical GPU: {}".format(len(GPUS)))

logical_gpus = tf.config.experimental.list_logical_devices("GPU") # 获取逻辑GPU，逻辑GPU就像电脑磁盘分区，默认是在第0个逻辑分区训练
print("Logical GPU: {}".format(len(logical_gpus)))

Physical GPU: 1
Logical GPU: 4


## 3. 测试

In [3]:
c = []
for gpu in logical_gpus:
    print(gpu.name)
    with tf.device(gpu.name):
        a = tf.constant([[1., 2., 3.], [4., 5., 6.]])
        b = tf.constant([[1., 2.], [3., 4.], [5., 6.]])
        c.append(tf.matmul(a, b))
with tf.device("/CPU:0"):
    matmul_sum = tf.add_n(c)

print(matmul_sum)

/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
/device:GPU:1
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:1
/device:GPU:2
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:2
/device:GPU:3
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:3
Executing op AddN in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[ 88. 112.]
 [196. 256.]], shape=(2, 2), dtype=float32)
